In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import re
import unicodedata
import sys

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import urllib.request
import urllib.response
import sys
import os, glob
import http.client, urllib
import json
import http.client, urllib.request, urllib.parse, urllib.error, base64

# Replace the accessKey string value with your valid access key.
accessKey = '82e4612615634c398bfd26e7d6327833'
url = 'westcentralus.api.cognitive.microsoft.com'
path = '/text/analytics/v2.0/keyPhrases'

def extract_keywords(body):
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = http.client.HTTPSConnection(url)
    body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
    body_json = json.dumps(body_req)
    conn.request ("POST", path, body_json, headers)
    response = conn.getresponse ()
    string = response.read().decode('utf-8')
    json_obj = json.loads(string)
    print("json_obj: ", json_obj)
    keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
    print("keyphrases_list: ", keyphrases_list)
    return keyphrases_list

In [3]:
# medical conditions training data
training_data = []

import csv

# adds every medical condition along with associated info from csv file
with open('Disease CSV.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            training_data.append({"class":row[1].strip(), "synonyms":row[2].strip(), 
                                  "sentence":row[3].strip(), "treatments":row[4].strip(), "danger level":row[5].strip()})
        line_count += 1

print ("%s total classes in training data" % len(training_data))

for term in training_data:
    print(term["class"] + " | " + term["synonyms"] + " | " + term["sentence"] + 
          " | " + term["treatments"] + " | " + term["danger level"])

28 total classes in training data
Common Cold | Upper Respiratory Tract;Nose and Throat Infection | Runny or stuffy nose;sore throat;cough;congestion;body aches;sneezing | Stay hydrated;rest;sooth a sore throat using saltwater gargle;take over the counter cold and cough medications | 1
Allergies | Allergic Reaction | Eye irritation;runny or stuffy nose;puffy watery eyes;sneezing;inflamed;itchy nose and throat | Remove the cause of allergy; 
otherwise take Antihistamines to relieve sneezing;Decongestants to relieve congestions in nasal membranes;Anti inflamary agents to reduce;Allergy shots | 1
Conjunctivitis | Pink Eye | Redness of eyes;itchy eyes;tearing or burning sensation of eyeball | See a doctor;Bacterial cases can be treated with antibiotic eye drops;Allergic reactions can be treated with other eye drops | 1
Irritable Bowel Syndrome (IBS) | Diarrhea | Watery, loose stools;frequent bowel movement;stomachache nausea;bloody stool | Diarrhea will typically go away in 2 days;Consult 

In [4]:
# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# method to remove punctuations from sentences.
def remove_punctuation(text):
     return text.translate(tbl)

# method to slice strings
def slicer(my_str,sub):
    index=my_str.find(sub)
    if index !=-1 :
         return my_str[index:] 
    else :
         raise Exception('Sub string not found!')

In [5]:
import string

words = []
words_total = []
classes = []
classes_details = []
documents = []
ignore_words = ['?']

# loop through each description per symptom
for pattern in training_data:
    symptoms = pattern['sentence']
    word_list = symptoms.split(';')
    print("WORD_LIST:",word_list)
    word_list_new = []
    for word in word_list:
        word = remove_punctuation(word)
        word = word.strip().lower()
        word_list_new.append(word)
    print("WORD_LIST_NEW:",word_list_new)
    joined_symps = ' '.join(s for s in word_list_new)
    print("joined_symps:",joined_symps)
    
    # tokenize each word in the description
    w = nltk.word_tokenize(joined_symps)
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        classes_details.append({"class":pattern['class'], "synonyms":pattern['synonyms'],
                                "treatments":pattern['treatments'], "danger level":pattern['danger level']})
    
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents", documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
print ("classes_details", classes_details)

WORD_LIST: ['Runny or stuffy nose', 'sore throat', 'cough', 'congestion', 'body aches', 'sneezing']
WORD_LIST_NEW: ['runny or stuffy nose', 'sore throat', 'cough', 'congestion', 'body aches', 'sneezing']
joined_symps: runny or stuffy nose sore throat cough congestion body aches sneezing
WORD_LIST: ['Eye irritation', 'runny or stuffy nose', 'puffy watery eyes', 'sneezing', 'inflamed', 'itchy nose and throat']
WORD_LIST_NEW: ['eye irritation', 'runny or stuffy nose', 'puffy watery eyes', 'sneezing', 'inflamed', 'itchy nose and throat']
joined_symps: eye irritation runny or stuffy nose puffy watery eyes sneezing inflamed itchy nose and throat
WORD_LIST: ['Redness of eyes', 'itchy eyes', 'tearing or burning sensation of eyeball']
WORD_LIST_NEW: ['redness of eyes', 'itchy eyes', 'tearing or burning sensation of eyeball']
joined_symps: redness of eyes itchy eyes tearing or burning sensation of eyeball
WORD_LIST: ['Watery, loose stools', 'frequent bowel movement', 'stomachache nausea', 'blood

In [6]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print("TOKENIZED WORDS: ",pattern_words)
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

TOKENIZED WORDS:  ['runny', 'or', 'stuffy', 'nose', 'sore', 'throat', 'cough', 'congestion', 'body', 'aches', 'sneezing']
TOKENIZED WORDS:  ['eye', 'irritation', 'runny', 'or', 'stuffy', 'nose', 'puffy', 'watery', 'eyes', 'sneezing', 'inflamed', 'itchy', 'nose', 'and', 'throat']
TOKENIZED WORDS:  ['redness', 'of', 'eyes', 'itchy', 'eyes', 'tearing', 'or', 'burning', 'sensation', 'of', 'eyeball']
TOKENIZED WORDS:  ['watery', 'loose', 'stools', 'frequent', 'bowel', 'movement', 'stomachache', 'nausea', 'bloody', 'stool']
TOKENIZED WORDS:  ['intense', 'pain', 'from', 'oneside', 'then', 'spreads', 'often', 'lasting', 'a', 'few', 'hours', 'accompanied', 'by', 'nausea']
TOKENIZED WORDS:  ['constant', 'dull', 'pain', 'of', 'head', 'can', 'last', 'from', 'half', 'an', 'hour', 'to', 'several', 'days']
TOKENIZED WORDS:  ['stomach', 'pain', 'due', 'to', 'gas', 'caused', 'by', 'swallowed', 'air', 'breakdown', 'of', 'undigested', 'food']
TOKENIZED WORDS:  ['abdominal', 'cramps', 'watery', 'diarrhea'

In [7]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=10000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 39999  | total loss: 0.61462 | time: 0.016s
| Adam | epoch: 10000 | loss: 0.61462 - acc: 0.9349 -- iter: 24/28
Training Step: 40000  | total loss: 0.55911 | time: 0.020s
| Adam | epoch: 10000 | loss: 0.55911 - acc: 0.9414 -- iter: 28/28
--
INFO:tensorflow:C:\Users\Steven\Documents\HTN PROJECT 2\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y, 'classes_details':classes_details}, open( "training_data", "wb" ) )